In [1]:
import wave #for getting the wav file
import matplotlib.pyplot as plt #for plotting
import librosa #for doing all of the wavestuff
import numpy as np #vector stuff
import pandas as pd #dataframe
import torch #tesnor storage of the matrix data
import os 
import pandas as pd
from scipy.io import wavfile

# All of the subroutines

In [2]:
#Gets basic stats of the audio wave
def get_mMa(clip):
    max_amp = np.max(clip)
    min_amp = np.min(clip)
    mean_amp = np.mean(clip) 
    med_amp = np.median(clip)
    return [max_amp, min_amp, mean_amp, med_amp]
#0) max signal amplitude, 1) min signal amplitude, 2) mean signal amplitude, 3) median signal amplitude

In [3]:
#Gets the fourier transform information
def get_fourier(clip, samp_rate):
    #Get the Fourier transform
    fourier = np.fft.fft(clip)
    #Get the spectrum
    magspectrum = np.abs(fourier)
    #get the frequencies that correspond to the fourier magnitudes
    frequencies = abs(np.fft.fftfreq(len(fourier), d=1/samp_rate)) 

    #get the f with max magnitude
    magmax = np.max(magspectrum) 
    #find the index
    fmaxindex = np.where(magspectrum == magmax)
    #get the corresponding frequency of maximum power at any point during the clip. If there are 2 points, pick one.
    maxpf = frequencies[int(np.max(fmaxindex[0]))] 

    #define max f as highest f with 1/3 of the peak frequency
    isrelmax = torch.tensor(magspectrum > magmax/3).to(torch.float)
    relfs = torch.tensor(frequencies)*isrelmax #frequencies over 1/3 max mult by 1, others zeroed out
    #maximum relevant frequency
    maxf = max(relfs).numpy() 
    #define min f as lowest f with 1/3 of the peak (may often be 0)
    minf = min(relfs).numpy()
    #the BW of the magnitudes is max-min
    bwfourier = maxf-minf
    fourierout = [magmax, maxpf, maxf.item(), minf.item(), bwfourier]
    return fourierout
#0) max magnitude of fourier, 1) f for max magnitude, 2) highest relevant f, 3) lowest relevant f, 4) range (max-min) relevant f

In [4]:
#get basic spectral stats
def get_spec(clip, hoplen, samp_rate, nfft, maxf):
    #Get BW
    bw = np.max(librosa.feature.spectral_bandwidth(y=clip, sr=samp_rate, hop_length=hoplen)[0]) #sometimes gives 2 items in vector, one can be 0

    
    #Get the spectral rolloff
    specroll = np.max(librosa.feature.spectral_rolloff(y= clip, sr=samp_rate, n_fft=nfft, hop_length=hoplen, roll_percent=0.85)[0])#sometimes gives 2 items in vector, one can be 0

    
    #get fundamental frequency
    try:
        f0 = np.nanmean(librosa.pyin(y=clip, fmin = 60, fmax = maxf, sr=samp_rate)[0])
    except RuntimeWarning:
    #sometimes it doesn't give a fundamental frequency and thinks it's nothing
        f0 = 0
        
    spect = [bw, specroll, f0]
    return spect
#0) spect energy BW, 1) spectral rolloff 85% energy, 2) fundamental freq (f0)

In [5]:
#get the frequencies with the max power (spectrogram) and spectral centroid
def get_fpwr(clip, nfft, hoplen, f_res, samp_rate):
    
    fourier = librosa.stft(clip, n_fft=nfft, hop_length=hoplen)
    #convert it from raw values to dB (log)
    findb = librosa.amplitude_to_db(abs(fourier))
    #0 is the frequency bins, 1 is the time stamp
    #frequency corresponding to each row
    f_bins = [i * f_res for i in range(fourier.shape[0])]
    
    #get the max energy value
    overallmaxdb = np.max(findb) 
    #find the row it's in
    overallindex = np.where(findb == overallmaxdb)
    #get the corresponding frequency for that row
    maxptf=f_bins[int(overallindex[0])] #frequency of maximum power at any point during the clip

    #sum up the energy for all time periods for each row
    rowsums = np.sum(fourier, axis=1)
    #get the max energy value
    rowmax = np.max(rowsums)
    #find the row it's in
    sumindex = np.where(rowsums == rowmax)
    #get the corresponding frequency for that row
    maxsumf=f_bins[int(sumindex[0])] #frequency of maximum power throughout the clip
    
    centroidvect = librosa.feature.spectral_centroid(y=clip, sr=samp_rate)
    meancent = np.mean(centroidvect)
    
    return [maxptf, maxsumf, meancent]
# 0) freq with max power at any point, 1) freq with max overall power, 2) spectral centroid

In [6]:
#Gets differences and gradients between clip segment data
def diffgrad(intensor):
    cols = intensor.shape[1]
    rows = intensor.shape[0]
    zerost = torch.zeros(1, cols)
    lastt = torch.cat((intensor, zerost), dim=0)
    nextt = torch.cat((zerost, intensor), dim=0)
    diffs = nextt-lastt
    grads = nextt/lastt #need to replace nans with 0s
    #remove the last line since it doesn't have meaning
    tensoroutput = torch.cat((diffs[0:rows], grads[0:rows]), dim=1)
    return tensoroutput

# Main Code

In [36]:
#Import the file 
#use Danny's subroutine not here to get all the filenames, this is just hardcoded for testing
file_path = 'testfileest.wav'

#Import it in librosa
audio_data, sample_rate = librosa.load(file_path, sr=None)
#basic file info
pts = len(audio_data)
secs = pts/sample_rate


#get data for the whole audio file************************

#get the wave stats
wavestats = get_mMa(audio_data)
#0) max signal amplitude, 1) min signal amplitude, 2) mean signal amplitude, 3) median signal amplitude

#get fourier stats
fourierstats = get_fourier(audio_data, sample_rate)
#0) max magnitude of fourier, 1) f for max magnitude, 2) highest relevant f, 3) lowest relevant f, 4) range (max-min) relevant f

#setupfor spectral features:
nfft = round(512*sample_rate/22050) #512 recommended for voice at 22050 Hz
hoplwhole = sample_rate*3 #when we want data for the whole thing
hopl = round(sample_rate/5) #number of audio samples between adjacent STFT columns, we want 200ms chunks for the clips
#frequency resolution
fres = sample_rate/nfft
#max frequency
maxf = fourierstats[2]

#get spectral stats
specstats = get_spec(audio_data, hoplwhole, sample_rate, nfft, maxf)
#0) spect energy BW, 1) spectral rolloff 85% energy, 2) fundamental freq (f0)

#get normalized range/f0
rangef0 = fourierstats[4]/specstats[2]

#get spectrogram stats
spectrostats = get_fpwr(audio_data, nfft, hoplwhole, fres, sample_rate)
# 0) freq with max power at any point, 1) freq with max overall power, 2) spectral centroid
#and normalize them
nspectrostats = [spectrostats[0]/spectrostats[2], spectrostats[1]/spectrostats[2]]

#get chroma data
chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate, hop_length=hoplwhole, n_fft=nfft)
#can give 1 or 2 columns. need to either max or avg them, not sure which makes sense yet

#get data for the different segments of the file************************

#Makes a setup vector for segmenting the audio file into subclips
n = 6 #number of sections to split into +1
#the shortest filess are 1 second and we don't want clips shorter than 200 ms since that's what the human ear picks up
pps = np.round(pts/n) #points per section, with the last section getting the extras or shorted
cutoffs = [int(pps * i) for i in range(0, n-1)]
cutoffs.append(int(pts))


#Run all of the subroutines to get the data for the segmented audio clips
segdata = []
for i in range(0, n-1):
    #make the shortened clip
    shortclip = audio_data[cutoffs[i]:cutoffs[i+1]]
    
    #run it through the subroutines in order to get the data for each
    clipdata= get_mMa(shortclip) #4 columns
    #add wave normalization:
    n1 = clipdata[0]/max(wavestats[0], np.abs(wavestats[1])) #to overall peak
    n2 = clipdata[1]/max(wavestats[0], np.abs(wavestats[1])) #to overall peak
    n3 = clipdata[2]/wavestats[2] #to overall avg mag
    n4 = clipdata[3]/wavestats[3] #to overall median mag
    clipndata = [n1, n2, n3, n4] #4 columns
    
    clipfourier = get_fourier(shortclip, sample_rate) #5 columns
    #add fourier normalization
    clipnfourier = [clipfourier[1]/fourierstats[1] , clipfourier[2]/fourierstats[2]] #2 columns, normalized to overall
    
    clipspec = get_spec(shortclip, hopl, sample_rate, nfft, maxf) #3 columns
    cliprangef0 = clipfourier[4]/clipspec[2] #single value
    clipspectro = get_fpwr(shortclip, nfft, hopl, fres, sample_rate) #3 columns
    #add spectral normalization:
    clipnspectro = [clipspectro[0]/clipspectro[2], clipspectro[1]/clipspectro[2]] #2 columns, normalized to f0 of clip
    
    
    #Outputs are all lists. Combine into 1 long row list and append as new row.
    segdata.append(clipdata + clipfourier + clipspec + [cliprangef0] + clipspectro + clipnspectro + clipndata + clipnfourier) #18 columns
    #0-3: wave stats, 4-8: fourier stats, 9-11: spectrum stats, 12: norm range, 
    #13-15: spectrogram stats, 16-17: normalized spectrogram, 
    #18-21: wave stats norm to overall, 22-23: fourier norm to overall
    
segmenttensor = torch.tensor(segdata)    


#get data for changes between segments************************

dg = diffgrad(segmenttensor[:,0:17]) #taking the diff of grad between data normalized to the overall would just be a linear combo of the data


#make one big output for the whole file*************************
#we have all of the data from the whole audio file, the data for the segments, and the data for the change between segments
#we want to flatten it into a single row
wholedata = torch.tensor(wavestats + fourierstats + specstats + [rangef0] + spectrostats + nspectrostats)
segmentdata = torch.reshape(torch.cat((segmenttensor, dg), dim=1), (1,290))[0]

final_output = torch.cat((wholedata, segmentdata), dim=0)

